In [2]:
import bs4
import requests
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import pandas as pd
import itertools
from ast import literal_eval
import ast
import math
import re
import matplotlib.pyplot as plt
from difflib import SequenceMatcher
from collections import Counter
import collections
import copy

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
urls_list = []

def create_list_urls(urls):
    
    for y in range(2023,2024):
        #create regular season urls
        for w in range(1,18):
            urls.append('https://www.nfl.com/injuries/league/' + str(y) + '/reg' + str(w))
        
        #create post season urls
        for i in range(1,5):
            urls.append('https://www.nfl.com/injuries/league/' + str(y) + '/post' + str(i))

        #create pro season urls
        urls.append('https://www.nfl.com/injuries/league/' + str(y) + '/pro1')    
            

    
def injury_data(url_list):
    all_players = []
    df_list = []
    df = pd.DataFrame()
    
    for i in url_list:
        print('Injuries from ' + i)
        curr_season = re.search("\d\d\d\d", i).group()
        try:
            alpha = requests.get(i)
            beta = bs4.BeautifulSoup(alpha.text)
            tables = beta.findAll("table")
            teams = beta.findAll("div", class_="d3-o-section-sub-title")
            idx = 0

            for table in tables:
                if table.findParent("table") is None:
                    tbody = table.find("tbody")
                    for tr in tbody:
                        try:
                            td = tr.find_all("td") 
                            rows = [tr.text.strip() for tr in td if tr is not None and len(tr) > 0]
                            for i in rows:
                                df_list.append(i) #at the player level
                            df_list.append(curr_season)
                            df_list.append(teams[idx].find("span").text)
                            all_players.append(copy.copy(df_list))
                            df_list = []
                    
                        except:
                            pass
                idx += 1        
                        
        except:
            print("Error with url: " + i)
        print(len(all_players))
    return all_players
        
        
create_list_urls(urls_list)
players_df = pd.DataFrame(injury_data(urls_list), columns = ['Player', 'Position', 'Injury', 'Practice', 'GameStatus', 'Season', 'Team'])

saved = players_df

Injuries from https://www.nfl.com/injuries/league/2023/reg1
176
Injuries from https://www.nfl.com/injuries/league/2023/reg2
390
Injuries from https://www.nfl.com/injuries/league/2023/reg3
644
Injuries from https://www.nfl.com/injuries/league/2023/reg4
922
Injuries from https://www.nfl.com/injuries/league/2023/reg5
1200
Injuries from https://www.nfl.com/injuries/league/2023/reg6
1511
Injuries from https://www.nfl.com/injuries/league/2023/reg7
1770
Injuries from https://www.nfl.com/injuries/league/2023/reg8
2088
Injuries from https://www.nfl.com/injuries/league/2023/reg9
2385
Injuries from https://www.nfl.com/injuries/league/2023/reg10
2667
Injuries from https://www.nfl.com/injuries/league/2023/reg11
2968
Injuries from https://www.nfl.com/injuries/league/2023/reg12
3299
Injuries from https://www.nfl.com/injuries/league/2023/reg13
3575
Injuries from https://www.nfl.com/injuries/league/2023/reg14
3904
Injuries from https://www.nfl.com/injuries/league/2023/reg15
4261
Injuries from https://w

In [12]:
counts = players_df.groupby(['Player', 'Season', 'Position', 'Team']).size().reset_index(name='TotalInjuryCount')
counts
sorted_df = counts.sort_values(by='TotalInjuryCount', ascending=False)
sorted_df

sorted_df.to_csv('totalInjury23.csv', index=False)

In [13]:
injury_counts = players_df.groupby(['Player', 'Season', 'Position', 'Team', 'Injury']).size().unstack(fill_value=0).reset_index()


injury_counts
filtered_df = injury_counts.filter(like='Concussion' )

concussion_sum = filtered_df.sum(axis=1)

# Add the sum as a new column to the original DataFrame
filtered_df['ConcussionTotal'] = concussion_sum

merged_df = pd.concat([counts, filtered_df['ConcussionTotal']], axis=1)
merged_df
concussion_sort = merged_df.sort_values(by='ConcussionTotal', ascending=False)
concussion_sort
concussion_sort.to_csv('totalConcussion23.csv', index=False)



In [14]:
practice_counts = players_df.groupby(['Player', 'Season', 'Position', 'Team', 'Practice']).size().unstack(fill_value=0).reset_index()
practice_counts.drop(practice_counts.columns[4], axis=1, inplace=True)
practice_counts
practice_counts.to_csv('practice23.csv', index=False)


In [15]:
status_counts = players_df.groupby(['Player', 'Season', 'Position','Team', 'GameStatus']).size().unstack(fill_value=0).reset_index()

status_counts.drop(status_counts.columns[4], axis=1, inplace=True)
status_counts
status_counts.to_csv('status23.csv', index=False)


In [ ]:
print("Complete")